<a href="https://colab.research.google.com/github/TsarIM/AEO-portfolio/blob/main/AEO_Portfolio_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [5]:
from google.colab import files
upload = files.upload()

Saving stockdata.csv to stockdata (1).csv


In [6]:
df=pd.read_csv('stockdata.csv')

In [7]:
display(df)

,MSFT,IBM,SBUX,AAPL,GSPC,Date
0,23.950705,80.517962,16.149666,11.086612,1416.599976,2007-01-03
1,23.910599,81.378851,16.167992,11.332687,1418.339966,2007-01-04
2,23.774242,80.642129,16.099269,11.251984,1409.709961,2007-01-05
3,24.006852,81.867244,16.039710,11.307550,1412.839966,2007-01-08
4,24.030914,82.835742,15.970989,12.246870,1412.109985,2007-01-09
...,...,...,...,...,...,...
2301,51.360001,132.800003,58.110001,96.099998,1929.800049,2016-02-24
2302,52.099998,134.500000,58.750000,96.760002,1951.699951,2016-02-25
2303,51.299999,132.029999,58.340000,96.910004,1948.050049,2016-02-26
2304,50.880001,131.029999,58.209999,96.690002,1932.229980,2016-02-29


In [8]:
data= df.drop(df.columns[[ -1]], axis=1)

In [9]:
display(data)

,MSFT,IBM,SBUX,AAPL,GSPC
0,23.950705,80.517962,16.149666,11.086612,1416.599976
1,23.910599,81.378851,16.167992,11.332687,1418.339966
2,23.774242,80.642129,16.099269,11.251984,1409.709961
3,24.006852,81.867244,16.039710,11.307550,1412.839966
4,24.030914,82.835742,15.970989,12.246870,1412.109985
...,...,...,...,...,...
2301,51.360001,132.800003,58.110001,96.099998,1929.800049
2302,52.099998,134.500000,58.750000,96.760002,1951.699951
2303,51.299999,132.029999,58.340000,96.910004,1948.050049
2304,50.880001,131.029999,58.209999,96.690002,1932.229980


In [10]:
covm=data.cov()
print(covm)

             MSFT          IBM         SBUX          AAPL           GSPC
MSFT    89.713806   170.643535   136.984573    284.871175    3161.510244
IBM    170.643535  1396.992250   364.135885    961.821166    7414.864627
SBUX   136.984573   364.135885   240.952168    497.630683    5020.519061
AAPL   284.871175   961.821166   497.630683   1200.295553   10380.736189
GSPC  3161.510244  7414.864627  5020.519061  10380.736189  130807.732047


In [11]:
Returns=data.pct_change().dropna()

print(Returns)

          MSFT       IBM      SBUX      AAPL      GSPC
1    -0.001675  0.010692  0.001135  0.022196  0.001228
2    -0.005703 -0.009053 -0.004251 -0.007121 -0.006085
3     0.009784  0.015192 -0.003699  0.004938  0.002220
4     0.001002  0.011830 -0.004284  0.083070 -0.000517
5    -0.010013 -0.011792 -0.003156  0.047856  0.001940
...        ...       ...       ...       ...       ...
2301  0.003517  0.003021 -0.005987  0.014891  0.004440
2302  0.014408  0.012801  0.011014  0.006868  0.011348
2303 -0.015355 -0.018364 -0.006979  0.001550 -0.001870
2304 -0.008187 -0.007574 -0.002228 -0.002270 -0.008121
2305  0.033412  0.025490  0.031438  0.039715  0.023869

[2305 rows x 5 columns]


In [12]:
random_weights = np.array(np.random.random(5))

weights_initial=random_weights/np.sum(random_weights)

print(weights_initial)

[0.19620626 0.30770207 0.28926838 0.14446648 0.0623568 ]


In [13]:
def sharpeRatio(Returns,unscaled_weights):

  weights=unscaled_weights/np.sum(unscaled_weights)

  P_return=np.sum((Returns.mean()*weights)*252)

  P_risk=np.sqrt(np.dot(weights.T,np.dot(Returns.cov()*252,weights)))

  return -P_return/P_risk

In [14]:
sharpeRatio(Returns,weights_initial)

-0.6806051258132959

In [16]:
def AEO(Returns,n_iters,n_popn):
  #initialise an ecosystem, population of potential solutions

  dim=len(Returns.columns)

  lb=np.full(dim,0.05) #lower bound
  ub=np.full(dim,1) #upper bound

  #constraints add krna bacha hai

  #for each individual calculate fitness and store in array
  individuals=np.zeros((n_popn,dim))
  fitness=np.zeros(n_popn)

  for i in range(n_popn):

    individuals[i,:]=np.random.rand(1,dim)*(ub-lb)+lb

    fitness[i]=sharpeRatio(Returns,individuals[i,:])

  #sort fitness in increasing order of sharpe ratio and get the indexes
  indF = np.argsort(fitness)[::-1]

  # Reorder PopPos and PopFit based on sorted indices
  individuals = individuals[indF, :]
  fitness = fitness[indF]

  # Select the best fitness and corresponding position
  BestF = fitness[-1]
  BestX = individuals[-1, :]

  #n_iters for loop

  new_indi=np.zeros((n_popn,dim))

#loop1
  for i in range(n_iters):

    #production
    r1=np.random.rand()
    a=(1-i/n_iters)*r1
    xrand = np.random.rand(dim)*(ub-lb)+lb
    new_indi[0,:]=(1-a)*individuals[n_popn-1,:]+a*xrand #eq 1

    #first herbivore
    u = np.random.randn(1,dim)
    v = np.random.randn(1,dim)
    # Compute C based on the given equation
    C = 0.5 * u / np.abs(v)
    # Update new_indi based on the given equation
    new_indi[1,:] = individuals[1,:]+C*(individuals[1,:]-new_indi[0,:]) #eq 6

#loop2
    for i in range (2,n_popn):
      u = np.random.randn(1,dim)
      v = np.random.randn(1,dim)
      C = 0.5 * u / np.abs(v)
      r=np.random.rand()

      if r<1/3:
        new_indi[i,:] = individuals[i,:]+C*(individuals[i,:]-new_indi[0,:]) #eq 6

      elif r>2/3:
        r2 = np.random.rand()
        rand_idx = np.random.randint(1,i)# [1,i)
        new_indi[i,:] = individuals[i,:]+C*(r2*(individuals[i,:]-new_indi[0,:])+(1-r2)*(individuals[i,:]-individuals[rand_idx,:])) #eq 8

      else:
        rand_idx2 = np.random.randint(1,i)
        new_indi[i,:] = individuals[i,:]+C*(individuals[i,:]-individuals[rand_idx2,:]) #eq 7
#loop2 end

      new_fit=np.zeros(n_popn)
#loop3
    for i in range(n_popn):
      new_indi[i,:] = spaceBound(new_indi[i,:],lb,ub) ##space limits
      new_fit[i] = sharpeRatio(Returns,new_indi[i,:])

      if new_fit[i]<fitness[i]:
        fitness[i]=new_fit[i]
        individuals[i,:]=new_indi[i,:]
#loop3 end

    mini = np.argmin(fitness)
#loop4
    for i in range(n_popn):
      r3 = np.random.rand()
      index = np.round(np.random.rand())+1
      D = 3*np.random.randn(1,dim)
      e = r3*(np.random.randint(1,3))-1
      h =2*r3-1
      new_indi[i,:] = individuals[mini,:]+D*(e*individuals[mini,:]-h*individuals[i,:]) #eq 9
#loop4 end

#loop5
    for i in range(n_popn):
      new_indi[i,:] = spaceBound(new_indi[i,:],lb,ub) ##space limits
      new_fit[i] = sharpeRatio(Returns,new_indi[i,:])

      if new_fit[i]<fitness[i]:
        fitness[i]=new_fit[i]
        individuals[i,:]=new_indi[i,:]
#loop5 end

    indF = np.argsort(fitness)[::-1]
    individuals = individuals[indF, :]
    fitness = fitness[indF]

    if fitness[-1]<BestF:
      BestF = fitness[-1]
      BestX = individuals[-1, :]

    print(BestX/np.sum(BestX),-BestF)

  return "optimal weights:",BestX/np.sum(BestX),-BestF



In [17]:
def spaceBound(x,lb,ub):
  for i in range(len(x)):
    if x[i]<lb[i] or x[i]>ub[i]:
      x[i]=np.random.uniform(lb[i],ub[i])
  return x

In [18]:
AEO(Returns,500,200)

[0.14111829 0.06300401 0.15549121 0.59299641 0.04739008] 0.8719453737205874
[0.0958399  0.0748263  0.1714574  0.61584626 0.04203013] 0.8793876779697859
[0.09791421 0.07306439 0.1862848  0.60769637 0.03504023] 0.8815578610941202
[0.05336162 0.0762686  0.19530133 0.63881593 0.03625253] 0.8868278184067937
[0.04546644 0.07718676 0.19157257 0.65103269 0.03474155] 0.8881968428547569
[0.04546644 0.07718676 0.19157257 0.65103269 0.03474155] 0.8881968428547569
[0.04288351 0.05247043 0.26607984 0.60129354 0.03727269] 0.8882380026320232
[0.03326575 0.04979817 0.26866705 0.61257663 0.0356924 ] 0.8904931698131334
[0.05526995 0.04498041 0.20974452 0.64971178 0.04029333] 0.8905806034371341
[0.05566131 0.03773289 0.19416542 0.67115011 0.04129027] 0.8916681733048885
[0.05566131 0.03773289 0.19416542 0.67115011 0.04129027] 0.8916681733048885
[0.04157362 0.03948663 0.1947385  0.68181696 0.04238429] 0.8924964603929824
[0.04480861 0.03865019 0.20045391 0.67930513 0.03678217] 0.8940744126080817
[0.03954462 

('optimal weights:',
 array([0.03364485, 0.03364485, 0.22616842, 0.67289702, 0.03364485]),
 0.896606456243397)